<a href="https://colab.research.google.com/github/nnnnmj/nnnnmj/blob/main/EM_NCPOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Install MOSEK if not already installed
!pip install mosek

SyntaxError: ignored

In [2]:
# Install ncpol2sdpa
!pip install ncpol2sdpa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ncpol2sdpa: filename=ncpol2sdpa-1.12.2-py3-none-any.whl size=70796 sha256=66acd8fe66ecfb9b22c7cebb033cc955fdd85504452c77a9d978d6d28b714049
  Stored in directory: /root/.cache/pip/wheels/5b/d7/fe/ab61f3bf30a350feab3bb4dccd63932d56cbbd32b9ec0d94fa
Successfully built ncpol2sdpa


In [3]:
import numpy as np
import pandas as pd
from math import sqrt
from copy import deepcopy
import matplotlib.pyplot as plt

from ncpol2sdpa import*

from sklearn.preprocessing import scale
from sklearn.metrics import f1_score
from scipy.io import arff

import time
import random

## Data Loader

In [7]:
trainpath = "ECG5000_TRAIN.arff"

In [5]:
def a2p(path):
    # load ARFF file
    data, meta = arff.loadarff(path)
    # turn to DataFrame
    df = pd.DataFrame(data)

    return df

In [8]:
train = a2p(trainpath)
train.target.value_counts()
train.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att132,att133,att134,att135,att136,att137,att138,att139,att140,target
0,-0.112522,-2.827204,-3.773897,-4.349751,-4.376041,-3.474986,-2.181408,-1.818286,-1.250522,-0.477492,...,0.792168,0.933541,0.796958,0.578621,0.257740,0.228077,0.123431,0.925286,0.193137,b'1'
1,-1.100878,-3.996840,-4.285843,-4.506579,-4.022377,-3.234368,-1.566126,-0.992258,-0.754680,0.042321,...,0.538356,0.656881,0.787490,0.724046,0.555784,0.476333,0.773820,1.119621,-1.436250,b'1'
2,-0.567088,-2.593450,-3.874230,-4.584095,-4.187449,-3.151462,-1.742940,-1.490659,-1.183580,-0.394229,...,0.886073,0.531452,0.311377,-0.021919,-0.713683,-0.532197,0.321097,0.904227,-0.421797,b'1'
3,0.490473,-1.914407,-3.616364,-4.318823,-4.268016,-3.881110,-2.993280,-1.671131,-1.333884,-0.965629,...,0.350816,0.499111,0.600345,0.842069,0.952074,0.990133,1.086798,1.403011,-0.383564,b'1'
4,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,-1.783423,-1.594450,-0.753199,...,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456,b'1'


## Function

### Uni-Model

In [9]:
def ncpop_UTS(K,L,T,level,MTS=False):
    # Define a function for solving the NCPOP problems with
    # given standard deviations of process noise and observtion noise,
    # length of estimation data and required relaxation level.

    N = len(K)
    M = len(L)

    # Decision Variables
    G = generate_operators("G", n_vars=1, hermitian=True, commutative=False)[0]
    Fdash = generate_operators("Fdash", n_vars=1, hermitian=True, commutative=False)[0]
    m = generate_operators("m", n_vars=T+1, hermitian=True, commutative=False)
    q = generate_operators("q", n_vars=T, hermitian=True, commutative=False)
    p = generate_operators("p", n_vars=T, hermitian=True, commutative=False)
    f = generate_operators("f", n_vars=T, hermitian=True, commutative=False)
    GG = generate_operators("GG", n_vars=1, hermitian=True, commutative=False)[0]
    FFdash = generate_operators("FFdash", n_vars=1, hermitian=True, commutative=False)[0]
    mm = generate_operators("mm", n_vars=T+1, hermitian=True, commutative=False)
    qq = generate_operators("qq", n_vars=T, hermitian=True, commutative=False)
    pp = generate_operators("pp", n_vars=T, hermitian=True, commutative=False)
    ff = generate_operators("ff", n_vars=T, hermitian=True, commutative=False)

    # Objective
    if MTS == False:
        print("MTS = False")
        obj = sum((K[n][i]-f[i])**2 for n in range(N) for i in range(T)) + 0.5*sum(p[i]**2 for i in range(T)) + 0.1*sum(q[i]**2 for i in range(T)) + sum((L[m][i]-ff[i])**2 for m in range (M) for i in range(T)) + 0.5*sum(pp[i]**2 for i in range(T)) + 0.1*sum(qq[i]**2 for i in range(T))
    else:
        obj = sum(sum((K[n][i]-f[i])**2 for n in range(N) for i in range(T)) + 0.0005*sum(p[i]**2 for i in range(T))+ 0.0001*sum(q[i]**2 for i in range(T))
                + sum((L[m][i]-ff[i])**2 for m in range (M) for i in range(T)) + 0.0005*sum(pp[i]**2 for i in range(T))+ 0.0001*sum(qq[i]**2 for i in range(T))
                )
    # Constraints
    ine1 = [f[i] - Fdash*m[i+1] - p[i] for i in range(T)]
    ine2 = [-f[i] + Fdash*m[i+1] + p[i] for i in range(T)]
    ine3 = [m[i+1] - G*m[i] - q[i] for i in range(T)]
    ine4 = [-m[i+1] + G*m[i] + q[i] for i in range(T)]
    #ine5 = [(K[i]-f[i])**2 for i in range(T)]
    ine5 = [ff[i] - FFdash*mm[i+1] - pp[i] for i in range(T)]
    ine6 = [-ff[i] + FFdash*mm[i+1] + pp[i] for i in range(T)]
    ine7 = [mm[i+1] - GG*mm[i] - qq[i] for i in range(T)]
    ine8 = [-mm[i+1] + GG*mm[i] + qq[i] for i in range(T)]
    ines = ine1+ine2+ine3+ine4+ine5+ine6+ine7+ine8

    # Solve the NCPO
    sdp = SdpRelaxation(variables = flatten([G,Fdash,f,p,m,q,GG,FFdash,ff,pp,mm,qq]),verbose = 1)
    sdp.get_relaxation(level, objective=obj, inequalities=ines)
    sdp.solve(solver='mosek')
    #print(sdp.primal, sdp.dual, sdp.status)
    if MTS == False:
        if (sdp[sum((K[n][i]-f[i])**2 for i in range(T) for n in range(N))] < 0 or sdp[sum((K[n][i]-ff[i])**2 for i in range(T) for n in range(N))] < 0):
            print("sum((K[i]-f[i])**2 for i in range(T)) < 0")
            return
    else:
        if (sdp[sum(sum((K[n][i]-f[i])**2 for n in range(N) for i in range(T)))] < 0 or
            sdp[sum(sum((L[m][i]-ff[i])**2 for m in range (M) for i in range(T)))] < 0):
            print("sum((K[i]-f[i])**2 for i in range(T)) < 0")
            return

    # nrmse_sim = 1-sqrt(sdp[sum((K[i]-f[i]+q[i])**2 for i in range(T))])/sqrt(sum((K[i]-np.mean(K))**2 for i in range(T)))

    if(sdp.status != 'infeasible'):
        pred_K = []
        pred_L = []
        params1 = sdp[G]
        params2 = (sdp[q[i]] for i in range(T))
        params3 = sdp[Fdash]
        # params4 = sdp[p]
        params5 = sdp[GG]
        # params6 = sdp[qq]
        params7 = sdp[FFdash]
        # params8 = sdp[pp]

        for i in range(T):
            pred_K.append(sdp[f[i]])
            pred_L.append(sdp[ff[i]])
        # params = [params1, params2, params3, params4, params5, params6, params7, params8]
        params = [params1, params2, params3, params5, params7]
        # params = [params1, params3, params5, params7]
        return pred_K, pred_L, params, sdp
    else:
        print('Cannot find feasible solution.')

### Multi-Model

In [ ]:
def ncpop_MTS(K,L,T,level=1):

    D = 2 # number of features, e.g., X.shape[1]

    # Decision operators
    G = generate_operators("G", n_vars=1, hermitian=True, commutative=False)[0]
    Fdash = generate_operators("Fdash", n_vars=D, hermitian=True, commutative=False)
    m = generate_operators("m", n_vars=T+1, hermitian=True, commutative=False)
    q = generate_operators("q", n_vars=T, hermitian=True, commutative=False)
    p = generate_operators("p", n_vars=T*D, hermitian=True, commutative=False)
    f = generate_operators("f", n_vars=T*D, hermitian=True, commutative=False)

    GG = generate_operators("GG", n_vars=1, hermitian=True, commutative=False)[0]
    FFdash = generate_operators("FFdash", n_vars=D, hermitian=True, commutative=False)
    mm = generate_operators("mm", n_vars=T+1, hermitian=True, commutative=False)
    qq = generate_operators("qq", n_vars=T, hermitian=True, commutative=False)
    pp = generate_operators("pp", n_vars=T*D, hermitian=True, commutative=False)
    ff = generate_operators("ff", n_vars=T*D, hermitian=True, commutative=False)

    # Objective
    obj = sum(K[i][t][0]-f[t]+K[i][t][1]-f[t+T] for t in range(T) for i in range(len(K)))+0.5*sum(
        (p[t]**2+p[t+T]**2) for t in range(T))+0.5*sum(q[t]**2 for t in range(T))+sum(
            L[i][t][0]-f[t]+L[i][t][1]-f[t+T] for t in range(T) for i in range(len(L)))+0.5*sum(
                (pp[t]**2+pp[t+T]**2) for t in range(T))+0.5*sum(qq[t]**2 for t in range(T))

    # Constraints
    ine1 = [f[i] - Fdash[0]*m[i+1] - p[i] for i in range(T)]
    ine2 = [-f[i] + Fdash[0]*m[i+1] + p[i] for i in range(T)]
    ine3 = [f[i+10] - Fdash[1]*m[i+1] - p[i+10] for i in range(T)]
    ine4 = [-f[i+10] + Fdash[1]*m[i+1] + p[i+10] for i in range(T)]
    ine5 = [m[i+1] - G*m[i] - q[i] for i in range(T)]
    ine6 = [-m[i+1] + G*m[i] + q[i] for i in range(T)]
    ine7 = [ff[i] - FFdash[0]*mm[i+1] - pp[i] for i in range(T)]
    ine8 = [-ff[i] + FFdash[0]*mm[i+1] + pp[i] for i in range(T)]
    ine9 = [ff[i+10] - FFdash[1]*mm[i+1] - pp[i+10] for i in range(T)]
    ine10 = [-ff[i+10] + FFdash[1]*mm[i+1] + pp[i+10] for i in range(T)]
    ine11 = [mm[i+1] - GG*mm[i] - qq[i] for i in range(T)]
    ine12 = [-mm[i+1] + GG*mm[i] + qq[i] for i in range(T)]


    ines = ine1+ine2+ine3+ine4+ine5+ine6+ine7+ine8+ine9+ine10+ine11+ine12

    # Solve the NCPOP
    sdp = SdpRelaxation(variables = flatten([G,Fdash,m,q,p,f,GG,FFdash,mm,qq,pp,ff]),verbose = 1)
    sdp.get_relaxation(level, objective=obj, inequalities=ines,chordal_extension=True)
    sdp.solve(solver='mosek')
    print(sdp.primal, sdp.dual, sdp.status)

    if(sdp.status != 'infeasible'):
        print('ok.')
        est_noise1 = []
        est_noise2 = []
        pred_Y = []
        pred_YY = []
        # params1 = sdp[G]
        # params2 = sdp[Fdash]
        for i in range(T):
            est_noise1.append(sdp[p[i]])
            est_noise2.append(sdp[q[i]])
            pred_Y.append(sdp[f[i]])
            pred_YY.append(sdp[ff[i]])
        for i in range(T):
            pred_Y.append(sdp[f[i+T]])
            pred_YY.append(sdp[ff[i+T]])
        # params = [params1, params2]
        params = 0
        est_noise = [est_noise2, est_noise1]
        return pred_Y, pred_YY, est_noise, sdp

    else:
        print('Cannot find feasible solution.')

## Test

In [16]:
/root/mosek/mosek.lic

NameError: ignored

In [13]:
X_1 = train[train.target==b'1'].iloc[:,:-1].values
X_2 = train[train.target==b'2'].iloc[:,:-1].values

f1_list_ncpop_ecg = []  # 5*50
duration_ncpop_ecg = []

# for t in [30,60,90,120,140]:
for t in [10]:

    t_start = time.time()
    for i in range(1):

        seed = i
        np.random.seed(seed)

        idx_1 = np.arange(len(X_1))
        idx_2 = np.arange(len(X_2))
        iidx_1 = random.sample(sorted(idx_1), 5)
        iidx_2 = random.sample(sorted(idx_2), 5)
        np.random.shuffle(iidx_1)
        np.random.shuffle(iidx_2)
        XX_1 = X_1[iidx_1]
        XX_2 = X_2[iidx_2]
        X = np.concatenate((XX_1, XX_2), axis=0).reshape(10,140)
        labels = np.concatenate((np.zeros(5),np.ones(5)),axis=0)

        win_size = t
        if win_size == 140:
            X = X
        else:
            s = random.sample(sorted(np.arange(140-win_size)),1)[0]
            X = X[:,s:(s+win_size)]
            print(s,X.shape)
##################################################
        np.random.seed(42)

        idx = np.random.randint(0,2,len(X))
        print(idx)
        a = np.where(idx==0)[0]
        b = np.where(idx==1)[0]
        print(a,b)
        K = [X[i].tolist() for i in a]  #idx = 0
        L = [X[i].tolist() for i in b]  #idx = 1
        print(len(K),len(L))

        for e in range(10):
            idx_ = deepcopy(idx)
            print(e, "start: label=", idx_)
            pred_k, pred_l, params, sdp_ = ncpop_UTS(K,L,t,level=1)

            for n in range(len(X)):
                cost_k = sum((X[n][i]-pred_k[i])**2 for i in range(t))
                cost_l = sum((X[n][i]-pred_l[i])**2 for i in range(t))
                print(cost_k, cost_l)
                if cost_k < cost_l: # K cluster --> 0
                    idx[n] = 0
                else:
                    idx[n] = 1

            if np.array_equal(idx_,idx):
                print(idx_, "end")
                break
            else:
                idx_ = deepcopy(idx)
                a = np.where(idx==0)[0]
                b = np.where(idx==1)[0]
                K = [X[i] for i in a]  #idx_in = 0
                L = [X[i] for i in b]  #idx_in = 1
                print(len(K),len(L))

        f1_1 = f1_score(labels, idx_)
        f1_2 = f1_score(labels, 1-idx_)

        if f1_1>f1_2:
            f1 = f1_1
        else:
            f1 = f1_2
        print('f1:',f1)

##################################################
        # f1 = f1_ncpop(X_train, X_label, seed=seed)
        # print(f1)
        f1_list_ncpop_ecg.append(f1)
    duration_ncpop_ecg.append(time.time()-t_start)

85 (10, 10)
[0 1 0 0 0 1 0 0 0 1]
[0 2 3 4 6 7 8] [1 5 9]
7 3
0 start: label= [0 1 0 0 0 1 0 0 0 1]
MTS = False
The problem has 86 noncommuting Hermitian variables
Calculating block structure...
Estimated number of SDP variables: 3827
Generating moment matrix...
Reduced number of SDP variables: 3827
Processing 26/80 constraints...

Processing 80/80 constraints...
MOSEK error 1008 (MSK_RES_ERR_MISSING_LICENSE_FILE): License cannot be located. The default search path is ':/root/mosek/mosek.lic:'.

*** A FLEXlm error occurred. FLEXlm reported:



*** end of FLEXlm report.



Error: ignored

## Result